<a href="https://colab.research.google.com/github/hirokimituya/book_create-profitable-AI-with-Python/blob/main/ch04_04_estimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.4 評価

In [1]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# 乱数の種
random_seed = 123

## 4.4.1 混同行列

In [3]:
from pickle import load
# データ読み込みからデータ分割まで

# ライブラリのimport
from sklearn.datasets import load_breast_cancer

# データのロード
cancer = load_breast_cancer()

# 入力データx
x = cancer.data

# 正解データy
# 良性: 0 悪性: 1に値を変換する
y = 1 - cancer.target

# 入力データを2次元に絞り込み
x2 = x[:, :2]

# (4)データ分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x2, y, train_size=0.7, test_size=0.3,
                                                    random_state=random_seed)

In [4]:
# アルゴリズム選択から評価まで

# アルゴリズム選択（ロジスティック回帰）
from sklearn.linear_model import LogisticRegression
algorithm = LogisticRegression(random_state=random_seed)

# 学習
algorithm.fit(x_train, y_train)

# 予測
y_pred = algorithm.predict(x_test)

# 評価
score = algorithm.score(x_test, y_test)

# 結果確認
print(f'score: {score:.4f}')

score: 0.8772


In [5]:
# 混同行列の計算

# 必要なライブラリの取り込み
from sklearn.metrics import confusion_matrix

# 混同行列の生成
# y_test: 検証データの正解データ
# y_pred: 検証データの予測結果
matrix = confusion_matrix(y_test, y_pred)

# 結果確認
print(matrix)

[[101   2]
 [ 19  49]]


In [6]:
# 混同行列表示用関数

def make_cm(matrix, columns):
    # matrix numpy配列

    # columns項目名リスト
    n = len(columns)

    # '正解データ'をn回繰り返すリスト生成
    act = ['正解データ'] * n
    pred = ['予測結果'] * n

    # データフレーム生成
    cm = pd.DataFrame(matrix, columns=[pred, columns], index=[act, columns])
    
    return cm

In [7]:
# make_cmを使った混同行列の表示
cm = make_cm(matrix, ['良性', '悪性'])
display(cm)

予測結果    
           良性  悪性
正解データ 良性  101   2
      悪性   19  49

## 4.4.2 精度・適合率・再現率・F値



* 精度（Accuracy） ... 「正解数」を「全体件数」で割った比率 (TP + TN) / (TP + FP + FN + TN)
* 適合率（Precision） ... モデルが「陽性」と予測した対象のうち、本当に「陽性」だった割合 TP / (TP + FP)
* 再現率（Recall） ... 本当に「陽性」だったデータのうち、どの程度をモデルで「陽性」と検知できたか TP / (TP + FN)
* F値（F-score） ... 適合率と再現率の調和平均 2 * (適合率) * (再現率) / ((適合率) + (再現率))




In [8]:
# 適合率・再現率・F値の計算

# ライブラリの取り込み
from sklearn.metrics import precision_recall_fscore_support

# 適合率・再現率・F値の計算
precision, recall, fscore, _ = precision_recall_fscore_support(
    y_test, y_pred, average='binary')

# 結果の確認
print(f'適合率: {precision:.4f}')
print(f'再現率: {recall:.4f}')
print(f'F値: {fscore:.4f}')

適合率: 0.9608
再現率: 0.7206
F値: 0.8235


## 4.4.3 確率値と閾値

In [10]:
# 確率値の取得
y_proba = algorithm.predict_proba(x_test)
print(y_proba[10:20, :])

[[0.7889 0.2111]
 [0.0812 0.9188]
 [0.8383 0.1617]
 [0.9391 0.0609]
 [0.369  0.631 ]
 [0.9451 0.0549]
 [0.9399 0.0601]
 [0.9494 0.0506]
 [0.9617 0.0383]
 [0.9836 0.0164]]


In [11]:
# positive(1)の確率値の取得
y_proba1 = y_proba[:, 1]

# 結果確認
print(y_test[10:20])
print(y_pred[10:20])
print(y_proba1[10:20])

[0 1 1 0 1 0 0 0 0 0]
[0 1 0 0 1 0 0 0 0 0]
[0.2111 0.9188 0.1617 0.0609 0.631  0.0549 0.0601 0.0506 0.0383 0.0164]


In [12]:
# 閾値を変化させる
thres = 0.5
print((y_proba1[10:20] > thres).astype(int))

thres = 0.7
print((y_proba1[10:20] > thres).astype(int))

[0 1 0 0 1 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]


In [13]:
# 閾値を変更した場合の予測関数の定義
def pred(algorithm, x, thres):
    # 確率値の取得（行列）
    y_proba = algorithm.predict_proba(x)

    # 予測結果1の確率値
    y_proba1 = y_proba[:, 1]

    # 予測結果1の確率 > 閾値
    y_pred = (y_proba1 > thres).astype(int)
    return y_pred

In [14]:
# 閾値0.5で予測結果取得
pred_05 = pred(algorithm, x_test, 0.5)

# 閾値0.7で予測結果取得
pred_07 = pred(algorithm, x_test, 0.7)

# 結果確認
print(pred_05[10:20])
print(pred_07[10:20])

[0 1 0 0 1 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]


## 4.4.4 PR曲線とROC曲線